In [8]:
import os
from keras.applications import MobileNet


In [9]:
# Mobile Net was designed on 224x224 image sizes
img_rows, img_cols = 224, 224

# Load the MobileNet model without FC or top layers
MobileNet = MobileNet(weights='imagenet',
                     include_top= False,
                     input_shape = (img_rows, img_cols, 3))

# Here we freeze the last 4 layers
# by defult Layers are set to trainable as True 
for layer in MobileNet.layers:
    layer.trainable = False

# Printing the layers
for (i, layer) in enumerate(MobileNet.layers):
    print(str(i)+" "+layer.__class__.__name__, layer.trainable)


0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

<h4>Function to return  </h4>

In [10]:
def addTopModelMobileNet(bottom_model, num_classes):
    
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation ='relu')(top_model)
    top_model = Dense(1024, activation ='relu')(top_model)
    top_model = Dense(512, activation ='relu')(top_model)
    top_model = Dense(num_classes, activation='softmax')(top_model)
    return top_model

<h4> Add FC head back onto MobileNet  </h4>

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPool2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model

# Number of classes:
num_classes = 10

FC_head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs= MobileNet.input, outputs=FC_head)

print(model.summary())

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

# Loading Dataset

In [15]:
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = '/home/iw/Documents/Gk/MasterOpenCV/Master OpenCV/monkey_breed/train'
validation_data_dir = '/home/iw/Documents/Gk/MasterOpenCV/Master OpenCV/monkey_breed/validation'

# Data Agumentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=45,
                                  width_shift_range=.3,
                                  height_shift_range=0.3,
                                  horizontal_flip=True,
                                  fill_mode='nearest'
                                  )

validation_datagen = ImageDataGenerator(rescale=1./255)

# Batch size
batch_size = 32

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                   target_size=(img_rows,img_cols),
                                                   batch_size=batch_size,
                                                   class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(validation_data_dir,
                                                             target_size=(img_rows,img_cols),
                                                             batch_size=batch_size,
                                                             class_mode='categorical')

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


# Training 

In [17]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint('/home/iw/Documents/Gk/MasterOpenCV/Master OpenCV/MonkeyBreedMobileNet.h5',
                            monitor="val_loss",
                            mode="min",
                            save_best_only= True,
                            verbose=1
                            )

earlystop = EarlyStopping(monitor='val_loss',
                         min_delta=0,
                         patience=3,
                         verbose=1,
                         restore_best_weights= True
                         )

# Put callback into a callback list
callbacks = [earlystop, checkpoint]


# Set learning rate
model.compile(loss='categorical_crossentropy',
             optimizer=RMSprop(lr=0.001),
             metrics=['accuracy'])


# Enter number of training and validation samples 
nb_train_sample = 1097
nb_validation_sample = 272

# Set Epochs
epochs = 5
batch_size = 16

history = model.fit_generator(train_generator,
                             steps_per_epoch=nb_train_sample//batch_size,
                             epochs=epochs,
                             callbacks=callbacks,
                             validation_data=validation_generator,
                             validation_steps=nb_validation_sample//batch_size)

Epoch 1/5
68/68 [==============================] - 57s 838ms/step - loss: 1.4746 - accuracy: 0.6116 - val_loss: 0.5641 - val_accuracy: 0.8314

Epoch 00001: val_loss improved from inf to 0.56413, saving model to /home/iw/Documents/Gk/MasterOpenCV/Master OpenCV/MonkeyBreedMobileNet.h5
Epoch 2/5
68/68 [==============================] - 61s 890ms/step - loss: 0.6578 - accuracy: 0.7894 - val_loss: 0.1942 - val_accuracy: 0.9062

Epoch 00002: val_loss improved from 0.56413 to 0.19421, saving model to /home/iw/Documents/Gk/MasterOpenCV/Master OpenCV/MonkeyBreedMobileNet.h5
Epoch 3/5
68/68 [==============================] - 60s 878ms/step - loss: 0.4755 - accuracy: 0.8513 - val_loss: 0.5662 - val_accuracy: 0.8535

Epoch 00003: val_loss did not improve from 0.19421
Epoch 4/5
68/68 [==============================] - 59s 863ms/step - loss: 0.4249 - accuracy: 0.8774 - val_loss: 0.4898 - val_accuracy: 0.8867

Epoch 00004: val_loss did not improve from 0.19421
Epoch 5/5
68/68 [=======================